In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Load data
credits_df = pd.read_csv('/content/tmdb_5000_credits.csv')
movies_df = pd.read_csv('/content/tmdb_5000_movies.csv')

## EDA and Preliminary Data Cleaning

In [7]:
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [9]:
print(credits_df.shape)
print(movies_df.shape)

(4803, 4)
(4803, 20)


In [282]:
# We want to merge these data frames on movie id column
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)
df = movies_df.merge(credits_df, on="id")

In [24]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [25]:
df.shape

(4803, 23)

In [13]:
# Dataframes merged successfully

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [283]:
# Drop unnecessary columns
df = df.drop(columns=['homepage', 
                      'production_countries', 
                      'status', 
                      'title_x', 
                      'title_y',
                      'spoken_languages'])
df.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,tagline,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [29]:
df.shape

(4803, 17)

In [284]:
# Notice some columns are in JSON format
# Must decode these into a list
import json

def json_to_string(col):
  df[col] = df[col].apply(json.loads)
  for index, i in zip(df.index, df[col]):
      ls = []
      for j in range(len(i)):
        ls.append((i[j]['name'])) # "name" contains => name of the genre
      df.loc[index, col] = str(ls)
  return df[col]

# For crew column only, we want to separate director from rest of crew
df['crew'] = df['crew'].apply(json.loads)
def json_to_string_crew(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']

df['crew'] = df['crew'].apply(json_to_string_crew)
df.rename(columns={'crew':'director'},inplace=True)

In [285]:
# Apply funtion to genre, keyword, prodction_company, cast, and crew columns
df['genres'] = json_to_string('genres')
df['keywords'] = json_to_string('keywords')
df['production_companies'] = json_to_string('production_companies')
df['cast'] = json_to_string('cast')

In [286]:
df.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,tagline,vote_average,vote_count,cast,director
0,237000000,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",19995,"['culture clash', 'future', 'space war', 'spac...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"['Ingenious Film Partners', 'Twentieth Century...",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",James Cameron
1,300000000,"['Adventure', 'Fantasy', 'Action']",285,"['ocean', 'drug abuse', 'exotic island', 'east...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",Gore Verbinski
2,245000000,"['Action', 'Adventure', 'Crime']",206647,"['spy', 'based on novel', 'secret agent', 'seq...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"['Columbia Pictures', 'Danjaq', 'B24']",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",Sam Mendes
3,250000000,"['Action', 'Crime', 'Drama', 'Thriller']",49026,"['dc comics', 'crime fighter', 'terrorist', 's...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"['Legendary Pictures', 'Warner Bros.', 'DC Ent...",2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"['Christian Bale', 'Michael Caine', 'Gary Oldm...",Christopher Nolan
4,260000000,"['Action', 'Adventure', 'Science Fiction']",49529,"['based on novel', 'mars', 'medallion', 'space...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,['Walt Disney Pictures'],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",Andrew Stanton


In [287]:
# Now cols will have entries with a list of a string of a string
# We want them as just strings
def list_to_string(col):
  df[col] = df[col].str.strip('[]').str.replace(' ','').str.replace("'",'')
  df[col] = df[col].str.split(',')
  return df[col]

In [288]:
df['genres'] = list_to_string('genres')
df['keywords'] = list_to_string('keywords')
df['production_companies'] = list_to_string('production_companies')
df['cast'] = list_to_string('cast')

In [289]:
df.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,tagline,vote_average,vote_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[IngeniousFilmPartners, TwentiethCenturyFoxFil...",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",James Cameron
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[WaltDisneyPictures, JerryBruckheimerFilms, Se...",2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",Gore Verbinski
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[ColumbiaPictures, Danjaq, B24]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",Sam Mendes
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[LegendaryPictures, WarnerBros., DCEntertainme...",2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",Christopher Nolan
4,260000000,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[WaltDisneyPictures],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",Andrew Stanton


In [291]:
# Notice that cast is listed in order of importance in the film
# We don't need all the info on the cast: let's just take the first 5 actors
df['cast'] = df['cast'].apply(lambda x: x[:5])

In [292]:
# Write function to convert lists of strings to just strings
def to_single_string(col):
  for index, ls in zip(df.index, df[col]):
    ls = ", ".join(ls)
    df[col][index] = ls
  return df[col]

In [293]:
df['genres'] = to_single_string('genres')
df['keywords'] = to_single_string('keywords')
df['production_companies'] = to_single_string('production_companies')
df['cast'] = to_single_string('cast')

In [294]:
df.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,tagline,vote_average,vote_count,cast,director
0,237000000,"Action, Adventure, Fantasy, ScienceFiction",19995,"cultureclash, future, spacewar, spacecolony, s...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"IngeniousFilmPartners, TwentiethCenturyFoxFilm...",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"SamWorthington, ZoeSaldana, SigourneyWeaver, S...",James Cameron
1,300000000,"Adventure, Fantasy, Action",285,"ocean, drugabuse, exoticisland, eastindiatradi...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"WaltDisneyPictures, JerryBruckheimerFilms, Sec...",2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"JohnnyDepp, OrlandoBloom, KeiraKnightley, Stel...",Gore Verbinski
2,245000000,"Action, Adventure, Crime",206647,"spy, basedonnovel, secretagent, sequel, mi6, b...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"ColumbiaPictures, Danjaq, B24",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"DanielCraig, ChristophWaltz, LéaSeydoux, Ralph...",Sam Mendes
3,250000000,"Action, Crime, Drama, Thriller",49026,"dccomics, crimefighter, terrorist, secretident...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"LegendaryPictures, WarnerBros., DCEntertainmen...",2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"ChristianBale, MichaelCaine, GaryOldman, AnneH...",Christopher Nolan
4,260000000,"Action, Adventure, ScienceFiction",49529,"basedonnovel, mars, medallion, spacetravel, pr...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,WaltDisneyPictures,2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"TaylorKitsch, LynnCollins, SamanthaMorton, Wil...",Andrew Stanton


In [295]:
# Get info on numeric features of dataset
df.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


## Text PreProcessing

In [296]:
# Select key features for model
FEATURES = ['genres',
            'keywords',
            'original_language',
            'overview',
            'tagline',
            'director',
            'cast']

In [297]:
# Fill any na's in feats columns withn empty strin
for feat in FEATURES:
  df[feat] = df[feat].fillna('')


We will use a bag of words to analyze the data

In [298]:
# Write function to create a bag of words using selcted features
def bag_of_words(row):
  bow = (row['genres'] + ' ' 
         + row['keywords'] + ' ' 
         + row['overview'] + ' ' 
         + row['tagline'] + ' '
         + row['director'] + ' '
         + row['cast'])
          #+ row['original_language'] + ' '
          #+ row['release_date'].tostring())
  return bow

In [299]:
df['bag_of_words'] = df.apply(bag_of_words, axis=1)
df['bag_of_words'][0]

'Action, Adventure, Fantasy, ScienceFiction cultureclash, future, spacewar, spacecolony, society, spacetravel, futuristic, romance, space, alien, tribe, alienplanet, cgi, marine, soldier, battle, loveaffair, antiwar, powerrelations, mindandsoul, 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Enter the World of Pandora. James Cameron SamWorthington, ZoeSaldana, SigourneyWeaver, StephenLang, MichelleRodriguez'

In [ ]:
# We see there are some unwanted symbols in our bag of words
# Let's clean the data for better processing
# We need to: Remove stopwords
# Remove all non-alphabetic text
# Expand Contractions
!pip install contractions
import contractions

STOPWORDS = stopwords.words('english')
def preprocess_text(text):
  # Remove all non-alphabetic text and replace with a space
  text = re.sub('[^A-z]', ' ', text)

  # Expand contractions
  text = contractions.fix(text)

  # Perfrom lemmatization 
  text = [word.lower() for word in text.split() if word not in STOPWORDS]

  return ' '.join([word for word in text])

In [301]:
# Apply text preprocessing function to bag of words col
df['bag_of_words'] = df['bag_of_words'].apply(lambda x: preprocess_text(x))
df['bag_of_words'][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul in nd century paraplegic marine dispatched moon pandora unique mission becomes torn following orders protecting alien civilization enter world pandora james cameron samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez'

Since we already have a keyword column, extracting keywords from the overview is unnecessary.

Additionally, since names are merged into one word, we do not need to take any further action to ensure the model will not confuse people with the same first OR last name. 

On a dataset where this hadn't been the case, we would want to perform these steps.

## Building the Recommendation Engine

We will analyze the similarity between films using [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) and use this to make targeted recommendations

In [302]:
# First we need to tokenize the data
# Use CounterVectorize
from sklearn.feature_extraction.text import CountVectorizer
tokenizer = CountVectorizer()
cv_matrix = tokenizer.fit_transform(df['bag_of_words'])

In [303]:
# Define the cosine similarity on the matrix 
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
cosine_sim = cosine_similarity(cv_matrix, cv_matrix)
print(cosine_sim)

[[1.         0.09012301 0.03746343 ... 0.03212463 0.01606231 0.        ]
 [0.09012301 1.         0.07698004 ... 0.03300492 0.01650246 0.        ]
 [0.03746343 0.07698004 1.         ... 0.01714986 0.03429972 0.01690309]
 ...
 [0.03212463 0.03300492 0.01714986 ... 1.         0.04411765 0.02898855]
 [0.01606231 0.01650246 0.03429972 ... 0.04411765 1.         0.0579771 ]
 [0.         0.         0.01690309 ... 0.02898855 0.0579771  1.        ]]


In [304]:
# Set indices to title so we can decode and 
# Map back from our matrix
indices = pd.Series(df['original_title'])

In [305]:
# Write function which, given a film,
# Returns 7 most similar films 
def get_recommendations(title, cosine_sim=cosine_sim):
  recommended = []
    
  try:
    film_index = indices[indices == title].index[0]   
    score = pd.Series(cosine_sim[film_index]).sort_values(ascending=False)
    top_7 = list(score.iloc[1:8].index) 
    # note index 0 is the input film
    
    for i in top_7:  
      recommended.append(list(df['original_title'])[i])
    
    return recommended
  except:
    print("Sorry, we don't know that film")

In [306]:
get_recommendations('Batman')

['Batman & Robin',
 'Batman Returns',
 'The Dark Knight Rises',
 'Batman Begins',
 'The Dark Knight',
 'Batman Forever',
 'Batman: The Dark Knight Returns, Part 2']

In [307]:
# Create an interface to prompt user to enter a film
def get_films():
  # Prompt user to enter a film title
    print("Welcome to the Film Recommendation Engine!")
    print()
  
    title = input("Please enter a film title, with proper capitalization and punctuation:\n\t")
    print()
  
    print("Thank you! You entered: " + title)
    print()
    
    while True:
      recommended = get_recommendations(title)
      if recommended is not None:
        print("Based off that, we think you might like the following films: ")
        return recommended
      else:
        print()
        title = input("Please try again, or press enter to exit: ")
        print()
        if not title:
          print("Thank you for using pur recommendation system!")
          print("See you next time!")
          break

In [308]:
get_films()

Welcome to the Film Recommendation Engine!

Please enter a film title, with proper capitalization and punctuation:
	The Smurfs

Thank you! You entered: The Smurfs

Based off that, we think you might like the following films: 


['The Smurfs 2',
 'The SpongeBob Movie: Sponge Out of Water',
 'The Croods',
 'Rise of the Guardians',
 'The Book of Life',
 'Aladdin',
 'Monster House']

In [310]:
get_films()

Welcome to the Film Recommendation Engine!

Please enter a film title, with proper capitalization and punctuation:
	Jason Bourne

Thank you! You entered: Jason Bourne

Based off that, we think you might like the following films: 


['The Bourne Ultimatum',
 'The Bourne Supremacy',
 'The November Man',
 'Safe House',
 'RED',
 'Malone',
 'Hitman']

In [311]:
get_films()

Welcome to the Film Recommendation Engine!

Please enter a film title, with proper capitalization and punctuation:
	Schindler's List

Thank you! You entered: Schindler's List

Based off that, we think you might like the following films: 


['Unbroken',
 'The Great Escape',
 'Red Tails',
 'Woman in Gold',
 'Saints and Soldiers',
 'Proud',
 'The Imitation Game']

Recommendations look pretty good!